In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from random import shuffle
from scipy.stats import entropy
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import nashpy as nash
import random
import tensorflow as tf
import six

import open_spiel
from open_spiel.python.algorithms import cfr
from open_spiel.python.algorithms import cfr_br_actions
from open_spiel.python.algorithms import exploitability
from open_spiel.python.algorithms import exploitability_br_actions
from open_spiel.python.algorithms import fictitious_play
from open_spiel.python.algorithms import deep_cfr

from open_spiel.python import policy

from open_spiel.python.algorithms import lp_solver
from open_spiel.python.algorithms import outcome_sampling_mccfr
from open_spiel.python.algorithms import external_sampling_mccfr
from open_spiel.python.algorithms import external_sampling_mccfr_br_actions
from open_spiel.python.algorithms import outcome_sampling_mccfr_br_actions
from open_spiel.python.algorithms import psro_oracle

from open_spiel.python.algorithms import best_response as pyspiel_best_response
import pyspiel
import time
from numpy import array

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
def _full_best_response_policy(br_infoset_dict):
    """Turns a dictionary of best response action selections into a full policy.

  Args:
    br_infoset_dict: A dictionary mapping information state to a best response
      action.

  Returns:
    A function `state` -> list of (action, prob)
  """

    def wrap(state):
        infostate_key = state.information_state_string(state.current_player())
        br_action = br_infoset_dict[infostate_key]
        ap_list = []
        for action in state.legal_actions():
            ap_list.append((action, 1.0 if action == br_action else 0.0))
        return ap_list

    return wrap

def _policy_dict_at_state(callable_policy, state):
    """Turns a policy function into a dictionary at a specific state.

  Args:
    callable_policy: A function from `state` -> lis of (action, prob),
    state: the specific state to extract the policy from.

  Returns:
    A dictionary of action -> prob at this state.
  """

    infostate_policy_list = callable_policy(state)
    infostate_policy = {}
    for ap in infostate_policy_list:
        infostate_policy[ap[0]] = ap[1]
    return infostate_policy


In [3]:
# game = pyspiel.load_game("leduc_poker_dummy", 
#                         {"players": pyspiel.GameParameter(2)})
game = pyspiel.load_game("kuhn_poker"
                         ,{"players": pyspiel.GameParameter(2)})


In [ ]:
# game = pyspiel.load_game_as_turn_based("markov_soccer", )

game = pyspiel.load_game("liars_dice", 
                        {"players": pyspiel.GameParameter(2)})
# matrix_kuhn = pyspiel.extensive_to_matrix_game(game)
# state = matrix_kuhn.new_initial_state()
# print(state.legal_actions(0))


#                          {"players": pyspiel.GameParameter(2)})

# game = turn_based_simultaneous_game(game=goofspiel(imp_info=True,num_cards=4,points_order=descending))

In [ ]:
dim = 50
payoffs = np.tril(np.random.uniform(-1, 1, (dim,dim)), -1)
payoffs = (payoffs - payoffs.T)
matrix_game = pyspiel.create_matrix_game(payoffs, -payoffs)
game = pyspiel.convert_to_turn_based(matrix_game)

In [ ]:
num_strats = 50
dummy_game_matrix = np.zeros((num_strats, num_strats))
for i in range(num_strats):
    for j in range(num_strats):
        dummy_game_matrix[i, j] = i-j
dummy_game = pyspiel.create_matrix_game(dummy_game_matrix, -dummy_game_matrix)
game = pyspiel.convert_to_turn_based(dummy_game)

In [ ]:
num_iterations = 58800
num_traversals = 180
with tf.Session() as sess:
    deep_cfr_solver = deep_cfr.DeepCFRSolver(
        sess,
        game,
        policy_network_layers=(32, 32),
        advantage_network_layers=(16, 16),
        num_iterations=num_iterations,
        num_traversals=num_traversals,
        learning_rate=1e-3,
        batch_size_advantage=None,
        batch_size_strategy=None,
        memory_capacity=1e7)
    sess.run(tf.global_variables_initializer())
#     for i in range(num_iterations):
    _, advantage_losses, policy_loss = deep_cfr_solver.solve()
#     if i%100 == 0:
#         print('iteration: ', i)
#             for player, losses in six.iteritems(advantage_losses):
#                 print("Advantage for player %d: %s", player,
#                              losses[:2] + ["..."] + losses[-2:])
#                 print("Advantage Buffer Size for player %s: '%s'", player,
#                              len(deep_cfr_solver.advantage_buffers[player]))
#             print("Strategy Buffer Size: '%s'",
#                          len(deep_cfr_solver.strategy_buffer))
#         print("Final policy loss: ", policy_loss)
#     if i % 100 == 0:
#         conv = exploitability.nash_conv(
#             game,
#             policy.PolicyFromCallable(game, deep_cfr_solver.action_probabilities))
#         print("Deep CFR - NashConv: ", conv)


In [ ]:
mccfr_es_solver = external_sampling_mccfr.ExternalSamplingSolver(game)
start_time = time.time()
mccfr_es_times = []
mccfr_es_exps = []
for i in range(200000000):
    mccfr_es_solver.iteration()
    if i % 100 == 0:
        conv = exploitability.exploitability(
            game,
            policy.PolicyFromCallable(game, mccfr_es_solver.callable_avg_policy()))

        print("Iteration: {} Conv: {}".format(i, conv))
        
#         br2 = exploitability.best_response(game, xfp_solver.average_policy_callable(), 0)
        
        elapsed_time = time.time() - start_time
        print(elapsed_time)
        mccfr_es_times.append(elapsed_time)
        mccfr_es_exps.append(conv)


In [ ]:
# np.save('./results/mccfr_es_liars_dice_times', np.array(mccfr_es_times))
# np.save('./results/mccfr_es_liars_dice_exps', np.array(mccfr_es_exps))

In [ ]:
brs = []
info_test = []
for i in range(2):
    mccfr_es_policy = policy.PolicyFromCallable(game, mccfr_es_solver.callable_avg_policy())
    br_info = exploitability.best_response(game, mccfr_es_policy, i)
    full_br_policy = _full_best_response_policy(br_info["best_response_action"])
    info_sets = br_info['info_sets']
    info_test.append(info_sets)
    brs.append(full_br_policy)


iters = 5000
br_list = [brs]
# br_list = []
info_set_list = []

start_time = time.time()
mccfr_es_psro_times = []
mccfr_es_psro_exps = []

br_conv_threshold = 10

for i in range(iters):
    print('Iteration: ', i)
    mccfr_es_br_solver = external_sampling_mccfr_br_actions.ExternalSamplingSolver(game, br_list)

    j = 0
    br_list_conv = 1000
    while br_list_conv > br_conv_threshold:
        j += 1
        mccfr_es_br_solver.iteration()
        if j%100==0:
#         if j%1==0:
#             print(j)
    
            mccfr_es_br_policy = policy.PolicyFromCallable(game, mccfr_es_br_solver.callable_avg_policy())
            br_list_conv = exploitability_br_actions.exploitability(game, br_list, 
                                                                    mccfr_es_br_policy)
            print("Br list conv: ", br_list_conv, j)
                
    mccfr_es_br_policy = policy.PolicyFromCallable(game, mccfr_es_br_solver.callable_avg_policy())
    conv = exploitability.exploitability(game, mccfr_es_br_policy)
#             print(cfr_br_solver.average_policy())
    if conv < br_conv_threshold:
        br_conv_threshold = br_conv_threshold*.5
        print('new br conv threshold: ', br_conv_threshold)

    print("Iteration {} exploitability {}".format(i, conv))
    elapsed_time = time.time() - start_time
    print('Total elapsed time: ', elapsed_time)
    mccfr_es_psro_times.append(elapsed_time)
    mccfr_es_psro_exps.append(conv)

    
    brs = []
    infos = []
    for i in range(2):
        mccfr_es_br_policy = policy.PolicyFromCallable(game, mccfr_es_br_solver.callable_avg_policy())
        br_info = exploitability.best_response(game, mccfr_es_br_policy, i)
        full_br_policy = _full_best_response_policy(br_info["best_response_action"])
        info_sets = br_info['info_sets']
        brs.append(full_br_policy)
        infos.append(info_sets)
        
    br_list.append(brs)
    info_set_list.append(infos)

In [ ]:
np.save('./results/mccfr_es_psro_leduc_dummy_times', np.array(mccfr_es_psro_times))
np.save('./results/mccfr_es_psro_leduc_dummy_exps', np.array(mccfr_es_psro_exps))

In [ ]:
mccfr_solver = outcome_sampling_mccfr.OutcomeSamplingSolver(game)
start_time = time.time()
mccfr_times = []
mccfr_exps = []
for i in range(2000000):
    mccfr_solver.iteration()
    if i % 100 == 0:
        conv = exploitability.exploitability(
            game,
            policy.PolicyFromCallable(game, mccfr_solver.callable_avg_policy()))

        print("Iteration: {} Conv: {}".format(i, conv))
        
#         br2 = exploitability.best_response(game, xfp_solver.average_policy_callable(), 0)
        
        elapsed_time = time.time() - start_time
        print(elapsed_time)
        mccfr_times.append(elapsed_time)
        mccfr_exps.append(conv)


In [ ]:
# np.save('./results/mccfr_leduc_dummy_times', np.array(mccfr_times))
# np.save('./results/mccfr_leduc_dummy_exps', np.array(mccfr_exps))

In [ ]:
brs = []
info_test = []
for i in range(2):
    mccfr_policy = policy.PolicyFromCallable(game, mccfr_solver.callable_avg_policy())
    br_info = exploitability.best_response(game, mccfr_policy, i)
    full_br_policy = _full_best_response_policy(br_info["best_response_action"])
    info_sets = br_info['info_sets']
    info_test.append(info_sets)
    brs.append(full_br_policy)


iters = 5000
br_list = [brs]
# br_list = []
info_set_list = []

start_time = time.time()
mccfr_psro_times = []
mccfr_psro_exps = []

br_conv_threshold = 10

for i in range(iters):
    print('Iteration: ', i)
    mccfr_br_solver = outcome_sampling_mccfr_br_actions.OutcomeSamplingSolver(game, br_list)

    j = 0
    br_list_conv = 1000
    while br_list_conv > br_conv_threshold:
        j += 1
        mccfr_br_solver.iteration()
        if j%100==0:
#         if j%1==0:
#             print(j)
    
            mccfr_br_policy = policy.PolicyFromCallable(game, mccfr_br_solver.callable_avg_policy())
            br_list_conv = exploitability_br_actions.exploitability(game, br_list, 
                                                                    mccfr_br_policy)
            print("Br list conv: ", br_list_conv, j)
                
    mccfr_br_policy = policy.PolicyFromCallable(game, mccfr_br_solver.callable_avg_policy())
    conv = exploitability.exploitability(game, mccfr_br_policy)
#             print(cfr_br_solver.average_policy())
    if conv < br_conv_threshold:
        br_conv_threshold = br_conv_threshold*.5
        print('new br conv threshold: ', br_conv_threshold)

    print("Iteration {} exploitability {}".format(i, conv))
    elapsed_time = time.time() - start_time
    print('Total elapsed time: ', elapsed_time)
    mccfr_psro_times.append(elapsed_time)
    mccfr_psro_exps.append(conv)

    
    brs = []
    infos = []
    for i in range(2):
        mccfr_policy = policy.PolicyFromCallable(game, mccfr_br_solver.callable_avg_policy())
        br_info = exploitability.best_response(game, mccfr_policy, i)
        full_br_policy = _full_best_response_policy(br_info["best_response_action"])
        info_sets = br_info['info_sets']
        brs.append(full_br_policy)
        infos.append(info_sets)
        
    br_list.append(brs)
    info_set_list.append(infos)

In [ ]:
np.save('./results/mccfr_psro_leduc_dummy_times', np.array(mccfr_psro_times))
np.save('./results/mccfr_psro_leduc_dummy_exps', np.array(mccfr_psro_exps))

In [ ]:
xfp_solver = fictitious_play.XFPSolver(game)

start_time = time.time()
xfp_times = []
xfp_exps = []
for i in range(40000):
    xfp_solver.iteration()
    if i % 100 == 0:
        conv = exploitability.exploitability(
            game,
            policy.PolicyFromCallable(game, xfp_solver.average_policy_callable()))

        print("Iteration: {} Conv: {}".format(i, conv))
        
#         br2 = exploitability.best_response(game, xfp_solver.average_policy_callable(), 0)
        
        elapsed_time = time.time() - start_time
        print(elapsed_time, 'elapsed time')
        xfp_times.append(elapsed_time)
        xfp_exps.append(conv)


In [ ]:
# np.save('./results/xfp_kuhn_times', np.array(xfp_times))
# np.save('./results/xfp_kuhn_exps', np.array(xfp_exps))

In [4]:
#                          {"players": pyspiel.GameParameter(2)})
cfr_solver = cfr.CFRSolver(game)

start_time = time.time()
times = []
exps = []
for i in range(1):
    cfr_solver.evaluate_and_update_policy()
    if i % 10 == 0:
        conv = exploitability.exploitability(game, cfr_solver.average_policy())
#         br = pyspiel_best_response.BestResponsePolicy(game, 0, cfr_solver.average_policy())
#         br2 = exploitability.best_response(game, cfr_solver.average_policy(), 0)
#         print(br)
#         print(br2)
        print("Iteration {} exploitability {}".format(i, conv))
        elapsed_time = time.time() - start_time
        print(elapsed_time)
        times.append(elapsed_time)
        exps.append(conv)


Iteration 0 exploitability 0.45833333333333326
0.021558284759521484


In [ ]:
# np.save('./results/cfr_leduc_dummy50_times', np.array(times))
# np.save('./results/cfr_leduc_dummy50_exps', np.array(exps))

In [ ]:
brs = []
info_test = []
for i in range(2):
    br_info = exploitability.best_response(game, cfr.CFRSolver(game).average_policy(), i)
    full_br_policy = _full_best_response_policy(br_info["best_response_action"])
    info_sets = br_info['info_sets']
    info_test.append(info_sets)
    brs.append(full_br_policy)

br_list = [[brs[0]], [1], [brs[1]], [1]]  
psro_solver = psro_oracle.PSRO(game, br_list)

psro_exps = []
psro_times = []
start_time = time.time()
for i in range(500):
    psro_solver.iteration()
    if i%2==0:
        conv = exploitability.exploitability(game, psro_solver._current_policy)
        print("exploitability: ", conv, i)
        elapsed_time = time.time() - start_time
        psro_times.append(elapsed_time)
        print(elapsed_time)
        psro_exps.append(conv)

In [ ]:
# np.save('./results/psro_leduc_dummy12_times', np.array(psro_times))
# np.save('./results/psro_leduc_dummy12_exps', np.array(psro_exps))

In [ ]:
brs = []
info_test = []
for i in range(2):
    br_info = exploitability.best_response(game, cfr.CFRSolver(game).average_policy(), i)
    full_br_policy = _full_best_response_policy(br_info["best_response_action"])
    info_sets = br_info['info_sets']
    info_test.append(info_sets)
    brs.append(full_br_policy)

In [7]:
brs = []
info_test = []
for i in range(2):
    br_info = exploitability.best_response(game, cfr_solver.average_policy(), i)
    full_br_policy = _full_best_response_policy(br_info["best_response_action"])
    info_sets = br_info['info_sets']
    info_test.append(info_sets)
    brs.append(full_br_policy)


iters = 3500
br_list = [brs]
# br_list = []
info_set_list = []

start_time = time.time()
cfr_psro_times = []
cfr_psro_exps = []

br_conv_threshold = 10

for i in range(iters):
    print('Iteration: ', i)
    cfr_br_solver = cfr_br_actions.CFRSolver(game, br_list)

#     if i < 4:
#         br_conv_threshold = 0.2
#     elif i < 10:
#         br_conv_threshold = 0.1
#     elif i < 20:
#         br_conv_threshold = 0.01
    
    j = 0
    br_list_conv = 1000
    while br_list_conv > br_conv_threshold:
        j += 1
        cfr_br_solver.evaluate_and_update_policy()
#         if j%10==0:
        if j%1==0:
#             print(j)
            br_list_conv = exploitability_br_actions.exploitability(game, br_list, cfr_br_solver.average_policy())
            print("Br list conv: ", br_list_conv, j)
            
    conv = exploitability.exploitability(game, cfr_br_solver.average_policy())
#             print(cfr_br_solver.average_policy())
    print("Iteration {} exploitability {}".format(i, conv))
    if conv < br_conv_threshold:
        br_conv_threshold = br_conv_threshold*.5
        print('new br conv threshold: ', br_conv_threshold)
        
    elapsed_time = time.time() - start_time
    print('Total elapsed time: ', elapsed_time)
    cfr_psro_times.append(elapsed_time)
    cfr_psro_exps.append(conv)

    
    brs = []
    infos = []
    for i in range(2):
        br_info = exploitability.best_response(game, cfr_br_solver.average_policy(), i)
        full_br_policy = _full_best_response_policy(br_info["best_response_action"])
        info_sets = br_info['info_sets']
        brs.append(full_br_policy)
        infos.append(info_sets)
        
    br_list.append(brs)
    info_set_list.append(infos)
    


Iteration:  0
Initialized
Br list conv:  0.0 1
Iteration 0 exploitability 0.45833333333333326
new br conv threshold:  5.0
Total elapsed time:  0.05467724800109863
Iteration:  1
Initialized
Br list conv:  0.25 1
Iteration 1 exploitability 0.25
new br conv threshold:  2.5
Total elapsed time:  0.10874462127685547
Iteration:  2
Initialized
Br list conv:  0.25 1
Iteration 2 exploitability 0.25
new br conv threshold:  1.25
Total elapsed time:  0.1615607738494873
Iteration:  3
Initialized
Br list conv:  0.25 1
Iteration 3 exploitability 0.25
new br conv threshold:  0.625
Total elapsed time:  0.24942779541015625
Iteration:  4
Initialized
Br list conv:  0.25 1
Iteration 4 exploitability 0.25
new br conv threshold:  0.3125
Total elapsed time:  0.32652831077575684
Iteration:  5
Initialized
Br list conv:  0.25 1
Iteration 5 exploitability 0.25
new br conv threshold:  0.15625
Total elapsed time:  0.4172954559326172
Iteration:  6
Initialized
Br list conv:  0.25 1
Br list conv:  0.12499999999999997 2

Br list conv:  0.01442164862906739 23
Br list conv:  0.014233375942072635 24
Br list conv:  0.012929560558621017 25
Br list conv:  0.01168819796767448 26
Br list conv:  0.011186666729200628 27
Br list conv:  0.01071492571595023 28
Br list conv:  0.01054411574705627 29
Br list conv:  0.01124858458118505 30
Br list conv:  0.010070470697360528 31
Br list conv:  0.009276050223241128 32
Br list conv:  0.010387500113721831 33
Br list conv:  0.011525103344772303 34
Br list conv:  0.013388745804506136 35
Br list conv:  0.013636823926621189 36
Br list conv:  0.011948389490485628 37
Br list conv:  0.009634224412263137 38
Br list conv:  0.007096418874614541 39
Br list conv:  0.007458618826868019 40
Br list conv:  0.00855387591317866 41
Br list conv:  0.008775715867548939 42
Br list conv:  0.008645185583325027 43
Br list conv:  0.009369223796660858 44
Br list conv:  0.009651553406427732 45
Br list conv:  0.009166916820750937 46
Br list conv:  0.007867145208680426 47
Br list conv:  0.00621869151485

Br list conv:  0.0029586869958585227 131
Br list conv:  0.002947799951564606 132
Br list conv:  0.00331652207568095 133
Br list conv:  0.003620990253293793 134
Br list conv:  0.0038445760600117895 135
Br list conv:  0.003972141525848405 136
Br list conv:  0.003989149969126832 137
Br list conv:  0.0038840836420981906 138
Br list conv:  0.0036535187609961217 139
Br list conv:  0.003347602999527549 140
Br list conv:  0.0031634192192454558 141
Br list conv:  0.0029607728148477175 142
Br list conv:  0.003309156707386046 143
Br list conv:  0.0035568635121955294 144
Br list conv:  0.003684326124935633 145
Br list conv:  0.0036935345173325917 146
Br list conv:  0.0035944830064406352 147
Br list conv:  0.003401230628765406 148
Br list conv:  0.0031305407515816175 149
Br list conv:  0.002801153162703407 150
Br list conv:  0.002432735130909358 151
Br list conv:  0.002044603338489076 152
Br list conv:  0.0019084378277350045 153
Br list conv:  0.0017218913565330007 154
Br list conv:  0.001699923614

KeyboardInterrupt: 

In [ ]:
# np.save('./results/cfr_psro_leduc_dummy50_times', np.array(cfr_psro_times))
# np.save('./results/cfr_psro_leduc_dummy50_exps', np.array(cfr_psro_exps))

In [ ]:
game_name = 'kuhn'
mccfr_times = np.load('./results/mccfr_'+game_name+'_times.npy')
mccfr_exps = np.load('./results/mccfr_'+game_name+'_exps.npy')
mccfr_psro_times = np.load('./results/mccfr_psro_'+game_name+'_times.npy')
mccfr_psro_exps = np.load('./results/mccfr_psro_'+game_name+'_exps.npy')

# xfp_times = np.load('./results/xfp_'+game_name+'_times.npy')
# xfp_exps = np.load('./results/xfp_'+game_name+'_exps.npy')
# psro_times = np.load('./results/psro_'+game_name+'_times.npy')
# psro_exps = np.load('./results/psro_'+game_name+'_exps.npy')
# cfr_psro_times = np.load('./results/cfr_psro_'+game_name+'_times.npy')
# cfr_psro_exps = np.load('./results/cfr_psro_'+game_name+'_exps.npy')
# times = np.load('./results/cfr_'+game_name+'_times.npy')
# exps = np.load('./results/cfr_'+game_name+'_exps.npy')

In [ ]:
game_name = 'leduc'
normal_times = np.load('./results/cfr_'+game_name+'_times.npy')
normal_exps = np.load('./results/cfr_'+game_name+'_exps.npy')
dummy50_times = np.load('./results/cfr_'+game_name+'_dummy50_times.npy')
dummy50_exps = np.load('./results/cfr_'+game_name+'_dummy50_exps.npy')
normal_cfr_psro_times = np.load('./results/cfr_psro_'+game_name+'_times.npy')
normal_cfr_psro_exps = np.load('./results/cfr_psro_'+game_name+'_exps.npy')
dummy50_cfr_psro_times = np.load('./results/cfr_psro_'+game_name+'_dummy50_times.npy')
dummy50_cfr_psro_exps = np.load('./results/cfr_psro_'+game_name+'_dummy50_exps.npy')

In [ ]:
game_name = 'leduc'
# mccfr_times = np.load('./results/mccfr_'+game_name+'_times.npy')
# mccfr_exps = np.load('./results/mccfr_'+game_name+'_exps.npy')
mccfr_es_times = np.load('./results/mccfr_es_'+game_name+'_times.npy')
mccfr_es_exps = np.load('./results/mccfr_es_'+game_name+'_exps.npy')

# mccfr_psro_times = np.load('./results/mccfr_psro_'+game_name+'_times.npy')
# mccfr_psro_exps = np.load('./results/mccfr_psro_'+game_name+'_exps.npy')


# xfp_times = np.load('./results/xfp_'+game_name+'_iters_4000000_times.npy')
# xfp_exps = np.load('./results/xfp_'+game_name+'_iters_4000000_exps.npy')
# # xfp_times = np.load('./results/xfp_'+game_name+'_iters_1000000every_5_times.npy')
# # xfp_exps = np.load('./results/xfp_'+game_name+'_iters_1000000every_5_exps.npy')

# psro_times = np.load('./results/psro_'+game_name+'_iters_1000000every_10_times.npy')
# psro_exps = np.load('./results/psro_'+game_name+'_iters_1000000every_10_exps.npy')

# # psro_times = np.load('./results/psro_'+game_name+'_iters_4000000_times.npy')
# # psro_exps = np.load('./results/psro_'+game_name+'_iters_4000000_exps.npy')


# cfr_psro_times = np.load('./results/cfr_psro_'+game_name+'_iters_4000000_times.npy')
# cfr_psro_exps = np.load('./results/cfr_psro_'+game_name+'_iters_4000000_exps.npy')
# times = np.load('./results/cfr_'+game_name+'_iters_times.npy')
# exps = np.load('./results/cfr_'+game_name+'_iters_exps.npy')

In [ ]:
plt.plot(mccfr_times, mccfr_exps, label='mccfr os')
plt.plot(mccfr_es_times, mccfr_es_exps, label='mccfr es')
# plt.plot(mccfr_psro_times, mccfr_psro_exps, label='mccfr-do')
plt.plot(times, exps, label='cfr')
# plt.plot(dummy50_times, dummy50_exps, label='cfr dummy 50')
# plt.plot(normal_times[:50], normal_exps[:50], label='cfr Normal')
# 
# plt.plot(xfp_times, xfp_exps, label='xfp')
# plt.plot(cfr_psro_times, cfr_psro_exps, label='cfr-do dummy 50')
# plt.plot(dummy50_cfr_psro_times, dummy50_cfr_psro_exps, label='cfr-do dummy 50')
# plt.plot(normal_cfr_psro_times[:10], normal_cfr_psro_exps[:10], label='cfr-do normal')

# plt.plot(psro_times[:15], psro_exps[:15], label='psro dummy 12')
plt.legend()
plt.yscale("log")
# plt.xscale('log')
plt.title('Leduc')
plt.ylabel('Exploitability')
plt.xlabel('Time (s)')
plt.show()

In [ ]:
plt.plot(mccfr_es_psro_times, mccfr_es_psro_exps, label='mccfr es psro')
plt.plot(mccfr_es_times, mccfr_es_exps, label='mccfr es')
plt.legend()
plt.show()

In [ ]:
mccfr_es_times

In [ ]:
cfr_br_solver = cfr_br_actions.CFRSolver(game, [brs])

for i in range(100):
    cfr_br_solver.evaluate_and_update_policy()
    if i % 10 == 0:
        conv = exploitability.exploitability(game, cfr_br_solver.average_policy())
        print("Iteration {} exploitability {}".format(i, conv))
